In [0]:
%tensorflow_version 2.x
import tensorflow as tf

# linear algebra
import numpy as np
# data manipulation
import pandas as pd
from sklearn.model_selection import train_test_split
# data visualisation
import matplotlib.pyplot as plt

# layers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Dense, AveragePooling2D
# model
from tensorflow.keras.models import Model

# dataset
import tensorflow_datasets as tfds

TensorFlow 2.x selected.


In [0]:
# load the data in
train_data, info = tfds.load('cats_vs_dogs', with_info = True, split = 'train')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete7LPUPS/cats_vs_dogs-train.tfrecord



Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [0]:
info

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    version=4.0.0,
    description='A large set of images of cats and dogs.There are 1738 corrupted images that are dropped.',
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    total_num_examples=23262,
    splits={
        'train': 23262,
    },
    supervised_keys=('image', 'label'),
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, Jared},
    title = {Asirra: A CAPTCHA that Exploits Interest-Aligned Manual Image Categorization},
    booktitle = {Proceedings of 14th ACM Conference on Computer and Communications Security (CCS)},
    ye

In [0]:
for i in train_data.take(1):
  print(i)

In [0]:
# resize the images to reduce the training time and improve the model efficiency
def resize(img):
  image_data = tf.image.resize(img['image'], [128,128])/255.0
  label = img['label']
  return {'image': image_data, 'label': label}

In [0]:
# load the data in
all_data = train_data.shuffle(2048).map(resize).batch(1000).prefetch(-1)
for data in all_data.take(1):
  X, Y = data['image'], data['label']
  print(X[0].shape) # shape of single image sample

(128, 128, 3)


In [0]:
# Build the model with VGG architecture, i'll use sparse categorical cross entropy as always

i = Input(shape = (128,128,3))
x = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dropout(.3)(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(.2)(x)
x = Dense(2, activation = 'softmax')(x) # we can use sigmoid because the labels are in binary but since softmax is kinda generalized sigmoid, i'll continue to use softmax

model = Model(i, x)

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])


In [0]:
# fitting the model using tensorflow data pipelines
# to train the model without exceeding the ram capacity, I'll train the model
# with batched data. Also apply the data augmentation for every batch

# TODO: I can use some callbacks to improve the efficiency and convenience,
#        something like LambdaCallback, ModelCheckpoint, EarlyStopping, CSVLogger would fit the project
for data in all_data.take(10):
  # split the data into 80% train and 20% test sets
  x_train, x_test, y_train, y_test = train_test_split(data['image'].numpy(), data['label'].numpy().flatten(), test_size = .2)
  # specifies # of batches of augmented data
  batch_size = 32
  # every parameter passed to ImageDataGenerator is hyperparameter so we can change it to improve the model accuracy
  data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range = .1, height_shift_range = .1, horizontal_flip = True, vertical_flip = True)
  # creating the generator using flow method
  train_generator = data_generator.flow(x_train, y_train, batch_size)
  steps_per_epoch = x_train.shape[0] // batch_size # using this formula is very convenient and used as default mostly
  # fit the model with augmented data using less epochs 
  r = model.fit(train_generator, validation_data = (x_test, y_test), epochs = 30)

In [0]:
model.save('cats_vs_dogs_classifier.h5') # I'll save the model because of the amount of current RAM won't be enough for doing further analysis

In [0]:
CNN = tf.keras.models.load_model('cats_vs_dogs_classifier.h5') # load the model again after restarted the notebook

In [0]:
CNN.summary() # model summary

In [0]:
# Evaluate the model with random selected data
for data in all_data.take(1):
  CNN.evaluate(data['image'], data['label'])

1000/1000 [==============================] - 17s 17ms/sample - loss: 0.4322 - accuracy: 0.8850
